## unittest 自动化测试框架
### 基本测试结构
> 单元测试包含两个部分：管理依赖关系、管理测试本身
单元测试类图：

![](https://images0.cnblogs.com/i/236038/201404/230040440455234.png)

In [1]:
# temp.py
import unittest

class SimpleisticTest(unittest.TestCase):
    
    def test(self):
        a = 'a'
        b = 'b'
        self.assertEqual(a,b)

### 运行测试
> **最简单的运行测试的方法是通过命令行界面**

```bash
ubuntu@VM-64-223-ubuntu:~/PylibNote$ python3 -m unittest temp.py 
.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
ubuntu@VM-64-223-ubuntu:~/PylibNote$ python3 -m unittest -v temp.py 
test (temp.SimpleisticTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK
```
> ** -v表示输出测试结果的细节 **


### 测试结果
> 测试可能有三种结果：  

- ok 测试通过
- FAIL 测试未通过，并抛出AssertionError异常
- ERROR 测试抛出AssertionError以外的其他异常

> 无法明确指定一个测试是否pass，测试状态只取决于异常的absense/presense

In [ ]:
# unittest_outcomes.py

import unittest

class OutcomesTest(unittest.TestCase):
    
    def testPass(self):
        return
    
    def testFail(self):
        self.assertFalse(True)
        
    def testError(self):
        raise RuntimeError('Test error')

> 为了让测试失败的原因更清晰，fail*()和assert*()方法都接收一个参数msg，可以产生更细致的错误消息

In [ ]:
import unittest


class FailureMessageTest(unittest.TestCase):
    
    def testFail(self):
        self.assertFalse(True, 'failure message goes here')

### 断言T/F
> 根据测试者的需求，有两种不同的方法写测试断言，来断言一个条件的真实性。

In [2]:
import unittest

class TruthTest(unittest.TestCase):
    
    def testAssertTrue(self):
        self.assertTrue(True)
        
    def testAssertFalse(self):
        self.assertFalse(False)
        

> 运行结果：  

```bash

testAssertFalse (temp2.TruthTest) ... ok
testAssertTrue (temp2.TruthTest) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK

```

### 测试相等
> 一种特殊案例是，单元测试包含了一个方法来测试两个值的相等性。

In [3]:
import unittest

class EqualityTest(unittest.TestCase):
    
    def testExpectEqual(self):
        self.assertEqual(1,3-2)
        
    def testExpectEqualFails(self):
        self.assertEqual(2, 3 - 2)

    def testExpectNotEqual(self):
        self.assertNotEqual(2, 3 - 2)

    def testExpectNotEqualFails(self):
        self.assertNotEqual(1, 3 - 2)

### 近似相等？
> 除了严格意义上的相等，也可以通过assertAlmostEqual()和assertNotAlmostEqual()测试两个浮点数之间的近似相等。  
包括两个参数：

- 用于比较的两个数
- 用于比较的小数位数

In [ ]:
import unittest


class AlmostEqualTest(unittest.TestCase):

    def testEqual(self):
        self.assertEqual(1.1, 3.3 - 2.2)

    def testAlmostEqual(self):
        self.assertAlmostEqual(1.1, 3.3 - 2.2, places=1)

    def testNotAlmostEqual(self):
        self.assertNotAlmostEqual(1.1, 3.3 - 2.0, places=1)

## 容器
> 除了通用的assertEqual()和assertNotEqual(),还有特殊的方法来比较两个容器，如(dict,list,set)

In [ ]:
import textwrap
import unittest


class ContainerEqualityTest(unittest.TestCase):

    def testCount(self):
        self.assertCountEqual(
            [1, 2, 3, 2],
            [1, 3, 2, 3],
        )

    def testDict(self):
        self.assertDictEqual(
            {'a': 1, 'b': 2},
            {'a': 1, 'b': 3},
        )

    def testList(self):
        self.assertListEqual(
            [1, 2, 3],
            [1, 3, 2],
        )

    def testMultiLineString(self):
        self.assertMultiLineEqual(
            textwrap.dedent("""
            This string
            has more than one
            line.
            """),
            textwrap.dedent("""
            This string has
            more than two
            lines.
            """),
        )

    def testSequence(self):
        self.assertSequenceEqual(
            [1, 2, 3],
            [1, 3, 2],
        )

    def testSet(self):
        self.assertSetEqual(
            set([1, 2, 3]),
            set([1, 3, 2, 4]),
        )

    def testTuple(self):
        self.assertTupleEqual(
            (1, 'a'),
            (1, 'b'),
        )

> 使用assertIn()来测试容器成员  

> 任何支持__in__操作符或者container的对象都可以使用assertIn()

In [ ]:
import unittest


class ContainerMembershipTest(unittest.TestCase):

    def testDict(self):
        self.assertIn(4, {1: 'a', 2: 'b', 3: 'c'})

    def testList(self):
        self.assertIn(4, [1, 2, 3])

    def testSet(self):
        self.assertIn(4, set([1, 2, 3]))

### 测试异常
> 如上所述，如果测试抛出AssertionError以外的异常，把它作为Error处理。对与处理测试中未覆盖的异常，这很有用。然而，在某些环境中，需要对测试中一些异常做特殊处理。相比如捕获异常，assertRaises()使得测试代码更清晰。比较如下两个案例：

In [ ]:
import unittest


def raises_error(*args, **kwds):
    raise ValueError('Invalid value: ' + str(args) + str(kwds))


class ExceptionTest(unittest.TestCase):

    def testTrapLocally(self):
        try:
            raises_error('a', b='c')
        except ValueError:
            pass
        else:
            self.fail('Did not see ValueError')

    def testAssertRaises(self):
        self.assertRaises(
            ValueError,
            raises_error,
            'a',
            b='c',
        )

> 两个测试结果是相同的，但是第二个测试使用了assertRaises()更加简洁。

### 测试装置
Fixtures是对一个测试用例环境的搭建和销毁的过程

> 夹具是测试所需的外部资源。 例如，一个类的测试可能都需要提供配置设置或其他共享资源的另一个类的实例。 其他测试装置包括数据库连接和临时文件（许多人会认为使用外部资源使得这些测试不是“单元”测试，但它们仍然是测试并且仍然有用）。

> unittest包含特殊的钩子来配置和清理测试所需的外部资源。 要为每个单独的测试用例建立外部资源，请重写TestCase上的setUp()方法。 要清理它们，请重写tearDown()方法。 要为测试类的所有实例管理一组外部资源，请为TestCase重写类方法setUpClass()和tearDownClass()。要处理模块中所有测试的外部资源，请使用模块级函数setUpModule（）和tearDownModule（）。

In [ ]:
import random
import unittest


def setUpModule():
    print('In setUpModule()')


def tearDownModule():
    print('In tearDownModule()')


class FixturesTest(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        print('In setUpClass()')
        cls.good_range = range(1, 10)

    @classmethod
    def tearDownClass(cls):
        print('In tearDownClass()')
        del cls.good_range

    def setUp(self):
        super().setUp()
        print('\nIn setUp()')
        # Pick a number sure to be in the range. The range is
        # defined as not including the "stop" value, so make
        # sure it is not included in the set of allowed values
        # for our choice.
        self.value = random.randint(
            self.good_range.start,
            self.good_range.stop - 1,
        )

    def tearDown(self):
        print('In tearDown()')
        del self.value
        super().tearDown()

    def test1(self):
        print('In test1()')
        self.assertIn(self.value, self.good_range)

    def test2(self):
        print('In test2()')
        self.assertIn(self.value, self.good_range)

> 运行结果：

```
In setUpModule()
In setUpClass()
test1 (temp4.FixturesTest) ... 
In setUp()
In test1()
In tearDown()
ok
test2 (temp4.FixturesTest) ... 
In setUp()
In test2()
In tearDown()
ok
In tearDownClass()
In tearDownModule()

----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK
```

> 当清理Fixtures时出错，tearDown()方法不一定会被调用。可以使用addCleanup()来确保Fixtures被正确释放。

In [ ]:
import random
import shutil
import tempfile
import unittest


def remove_tmpdir(dirname):
    print('In remove_tmpdir()')
    shutil.rmtree(dirname)


class FixturesTest(unittest.TestCase):

    def setUp(self):
        super().setUp()
        self.tmpdir = tempfile.mkdtemp()
        self.addCleanup(remove_tmpdir, self.tmpdir)

    def test1(self):
        print('\nIn test1()')

    def test2(self):
        print('\nIn test2()')

> 例子创建了一个临时目录，当测试完成，然后使用shutil清理掉

### 以不同的输入重复测试
> 使用相同测试逻辑测试不同输入很常用。 与其为每个小案例定义单独的测试方法，一个常见的方法是使用一个包含几个相关断言调用的测试方法。 这种方法的问题是，只要一个断言失败，其余的就会被跳过。 更好的解决方案是使用subTest（）为测试方法中的测试创建上下文。 如果测试失败，则报告失败并继续进行其余的测试。

In [5]:
import unittest


class SubTest(unittest.TestCase):

    def test_combined(self):
        self.assertRegex('abc', 'a')
        self.assertRegex('abc', 'B')
        # The next assertions are not verified!
        self.assertRegex('abc', 'c')
        self.assertRegex('abc', 'd')

    def test_with_subtest(self):
        for pat in ['a', 'B', 'c', 'd']:
            with self.subTest(pattern=pat):
                self.assertRegex('abc', pat)

> 上例中，test_combined() 方法不会对'c'和'd'执行测试，而test_with_subtest()，会测试所有测试用例。注意test runner只认为有两个测试案例。

## 跳过测试
> 如果不符合某些外部条件，能够跳过测试通常很有用。 例如，当编写测试来检查特定版本的Python下的库的行为时，没有理由在其他版本的Python下运行这些测试。 测试类和方法可以使用skip（）进行修饰，以便始终跳过测试。 装饰skipIf（）和skipUnless（）可用于在跳过之前检查条件。

In [6]:
import sys
import unittest


class SkippingTest(unittest.TestCase):

    @unittest.skip('always skipped')
    def test(self):
        self.assertTrue(False)

    @unittest.skipIf(sys.version_info[0] > 2,
                     'only runs on python 2')
    def test_python2_only(self):
        self.assertTrue(False)

    @unittest.skipUnless(sys.platform == 'Darwin',
                         'only runs on macOS')
    def test_macos_only(self):
        self.assertTrue(True)

    def test_raise_skiptest(self):
        raise unittest.SkipTest('skipping via exception')


> 对于难以在单个表达式中传递给skipIf（）或skipUnless（）的复杂条件，测试用例可能会直接引发SkipTest以使测试被跳过。

### 忽略失败测试
> 与其删除预计会失败的测试用例，不如使用expectedFailure()装饰，该失败就会被忽略。


> 如果一项预计会失败的测试事实上已经通过，那么该条件将被视为一种特殊的失败并报告为“意想不到的成功”。

In [ ]:
import unittest


class Test(unittest.TestCase):

    @unittest.expectedFailure
    def test_never_passes(self):
        self.assertTrue(False)

    @unittest.expectedFailure
    def test_always_passes(self):
        self.assertTrue(True)